In [1]:
import numpy as np
import pandas as pd
import email
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
from email.parser import BytesParser
from email.policy import default

import csv
import os
import pathlib
import pprint

In [22]:
__file__ = "/Users/satvikajmera/Documents/SMU MSDS Course Material/05-Summer 2022/DS 7333 - QtW/Case Studies - on file/Case Study 03/Satvik/"

emails = {"index":[]}
for path, subdirectories, filelist in os.walk(pathlib.Path(__file__) / "SpamAssassinMessages"):
    for f in filelist:
        
        label = 0 if "ham" in path.lower() else 1
        with open(os.path.join(path, f), "rb") as fp:
            msg = BytesParser(policy=default).parse(fp)
            with open(os.path.join(path, f), "r",encoding= 'utf-8', errors='ignore') as new_f:
                msg = email.message_from_file(new_f)
                for part in msg.walk():
                    if part.get_content_type() == 'text/plain':
                        body = part.get_payload()  # decode
                emails["index"].append({
                    "Subject": msg.get("Subject"),
                    "From": msg.get("from"),
                    "Body": body,
                    "isSpam": label,
                    "FileID": f
                })
cols = ["Subject", "From", "Body", "isSpam", "FileID"]
try:
    with open(pathlib.Path(__file__).parent / "test.csv", 'w', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["Subject","From","Body","isSpam","FileID"])
        writer.writeheader()
        for data in emails["index"]:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [23]:
pd.set_option('display.max_colwidth', None)

In [24]:
df = pd.read_csv("../test.csv", header=0)

In [25]:
df

,Subject,From,Body,isSpam,FileID
0,NaN,NaN,NaN,1,.DS_Store
1,Let us find the right mortgage lender for you AFPE,pamela4701@eudoramail.com,"Dear Homeowner,\n \nInterest Rates are at their lowest point in 40 years!\n\nWe help you find the best rate for your situation by\nmatching your needs with hundreds of lenders!\n\nHome Improvement, Refinance, Second Mortgage,\nHome Equity Loans, and More! Even with less than\nperfect credit!\n\nThis service is 100% FREE to home owners and new\nhome buyers without any obligation. \n\nJust fill out a quick, simple form and jump-start\nyour future plans today!\n\n\nVisit http://61.145.116.186/user0201/index.asp?Afft=QM10\n\n\n\n\n\n\nTo unsubscribe, please visit:\n\nhttp://61.145.116.186/light/watch.asp\n\n\n",1,00249.5f45607c1bffe89f60ba1ec9f878039a
2,"Friend, Copy ANY DVD or Playstation Game with this software......",CopyYourDVD <atomica2020@hotmail.com>,"Friend,Now you can copy DVD's and Games\nhttp://www.terra.es/personal9/iop1008/\n\nBACKUP DVD VIDEO's WITH YOUR CD-R BURNER\n\nWith 321 studio's software, you can now copy\nany DVD and Playstation Game. Never buy another \nbackup DVD movie again. Just copy it!\n\nThis is the first time this software is being made\navailable to the public. All the software you need \nto burn your own DVD Video, is included in 321 Studio's \nsoftware package DVD Copy Plus! The movies will play \nin a standard DVD player. With detailed, easy to follow, \nstep-by-step instructions, you can BURN your own DVD \nVideo using nothing more than your DVD-ROM \nand CD-R drives. Purchase a copy! Click below.\n\nhttp://www.terra.es/personal9/iop1008/\n\nOrder today and receive!\n\n*Step by Step Interactive Instructions\n*All Software Tools Included On CD\n*No DVD Burner Required\n*FREE Live Technical Support\n*30 Day Risk Free Trial Available\n\nWith DVD Copy Plus you can backup Your DVD Movies with \nthe same 74min or 80min CD-R's you've used in the past \nto create audio CD's. Our software compresses the large \nDVD files on your standard DVD to VCD, SVCD, and DivX \nmuch the same way the popular MP3 format compresses audio. \nOrder today and start burning\nhttp://www.terra.es/personal9/iop1008/\n\nThank You,\n\nCopymyDVD\n\n\nhttp://inglesa.net/unsub.php?client=atomicDOT\n\n\n\nWe take your privacy very seriously and it is our policy never to send\nunwanted email messages. This message has been sent to zzzz@example.com\nbecause you originally joined one of our member sites or you signed up\nwith a party that has contracted with atomicDOT. Please\nhttp://tilw.net/unsub.php?client=atomicDOT&msgid=17090200005\nto Unsubscribe (replying to this email WILL NOT unsubscribe you). \n\n\n\n\n\nTRCK:atomicDOT;mp*qhwqrwhlqf!frp;8;\n\n\n",1,0355.94ebf637e4bd3db8a81c8ce68ecf681d
3,5% Guaranteed for Eight Years,"""IQ - Safe Harbor"" <sh@insiq.us>","Pocket the newest 8 year annuity!\t Pocket the newest 8 year\nannuity!\t\n \t ...from Safe Harbor Financial\t\n \t5% Guaranteed for 8 Years (effective yield)\t \n \t8 Year Surrender Period\t \t Please fax back contracts with\na copy of your insurance\nlicense to 215-564-0155.\nClick Here to Contract <http://iiq.us/images/sh/0903/sh_mony_app.pdf> \t\n \tIssued to Age 90\t \n \tA Rated Company\t \n \tAlternative 10 Year Guaranteed Plan\t \n\nCall today for more information!\n \n- or -\n\nPlease fill out the form below for more information\t \nName:\t \t \t\nAddress:\t \t \nCity:\t State: Zip: \t\nPhone:\t Fax: \t\nE-mail:\t \t \n \t \t\n \n \nWe don't want anyone to receive our mailings who does not wish to\nreceive them. This is a professional communication sent to insurance\nprofessionals. To be removed from this mailing list, DO NOT REPLY to\nthis message. Instead, go here: http://www.insuranceiq.com/optout\n<http://www.insuranceiq.com/optout/> \n\nLegal Notice <http://www.insiq.com/legal.htm> \n",1,0395.bb934e8b4c39d5eab38f828a26f760b4
4,Congratulations! You Get a Free Handheld Organizer!,Customer Service <greatoffers@sendgreatoffers.co

In [3]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

Build a spam classifier using naive Bayes and clustering. You will have to create your own dataset from the input messages. Be sure to document how you created your dataset.

In [4]:
df = pd.read_csv("../test.csv", header=0)

In [5]:
df["isSpam"].value_counts()

0    6954
1    2399
Name: isSpam, dtype: int64

In [6]:
df.isna().sum()

Subject    25
From        6
Body        0
isSpam      0
FileID      0
dtype: int64

In [10]:
df

Subject  \
0               Let us find the right mortgage lender for you      AFPE   
1     Friend, Copy ANY DVD or Playstation Game with this software......   
2                                         5% Guaranteed for Eight Years   
3                   Congratulations! You Get a Free Handheld Organizer!   
4                            PROTECT YOUR INFORMATION AND YOUR COMPUTER   
...                                                                 ...   
9348         RE: [Razor-users] What's wrong with the Razor servers now?   
9349                          Re: W3C approves HTML 4 'emotitags' [...]   
9350                                              Re: Ximian apt repos?   
9351                                                 Re: Installing RPM   
9352                        scan bug: no update after Pick's "New FTOC"   

                                                    From  \
0                              pamela4701@eudoramail.com   
1                  CopyYourDVD <atomica2020@hotmail.com>   
2                         IQ - Safe Harbor <sh@insiq.us>   
3     Customer Service <greatoffers@sendgreatoffers.com>   
4                                don <apf@wu-wien.ac.at>   
...                                                  ...   
9348                 "Rose, Bobby" <brose@med.wayne.edu>   
9349                                    harley@argote.ch   
9350                    Mark Derricutt <mark@talios.com>   
9351                   Matthias Saou <matthias@egwn.net>   
9352                     Brent Welch <welch@panasas.com>   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [12]:
df = df.dropna(how='any',axis=0) 

In [13]:
df.isna().sum()

Subject    0
From       0
Body       0
isSpam     0
dtype: int64

In [15]:
df["isSpam"].value_counts()

0    6944
1    2383
Name: isSpam, dtype: int64

In [16]:
#remove new lines and such
df = df.replace(r'\\n',' ', regex=True)
df = df.replace(r'\\t',' ', regex=True)
df = df.replace(r'b\'',' ', regex=True)
df = df.replace(r'b\"',' ', regex=True)


,Subject,From,Body,isSpam
9349,RE: [Razor-users] What's wrong with the Razor ...,"""Rose, Bobby"" <brose@med.wayne.edu>","I'm one of the 30,000 but it's not working ve...",0
9350,Re: W3C approves HTML 4 'emotitags' [...],harley@argote.ch,Damien Morton quoted: >W3C approves HTML 4 'e...,0
9351,Re: Ximian apt repos?,Mark Derricutt <mark@talios.com>,"On Mon, 2002-07-22 at 06:50, che wrote: > th...",0
9352,Re: Installing RPM,Matthias Saou <matthias@egwn.net>,"Once upon a time, Manfred wrote : > I would ...",0
9353,"scan bug: no update after Pick's ""New FTOC""",Brent Welch <welch@panasas.com>,"If you run Pick, and then use the ""New FTOC"" ...",0


In [8]:
df

,Subject,From,Body,isSpam,FileID
0,Let us find the right mortgage lender for you ...,pamela4701@eudoramail.com,"b'Dear Homeowner,\n \nInterest Rates are at th...",1,00249.5f45607c1bffe89f60ba1ec9f878039a
1,"Friend, Copy ANY DVD or Playstation Game with ...",CopyYourDVD <atomica2020@hotmail.com>,"b""Friend,Now you can copy DVD's and Games\nhtt...",1,0355.94ebf637e4bd3db8a81c8ce68ecf681d
2,5% Guaranteed for Eight Years,IQ - Safe Harbor <sh@insiq.us>,"b"" Pocket the newest 8 year annuity!\t Pocket ...",1,0395.bb934e8b4c39d5eab38f828a26f760b4
3,Congratulations! You Get a Free Handheld Organ...,Customer Service <greatoffers@sendgreatoffers....,b'<html><head>\n<title>Congratulations! You Ge...,1,0485.9021367278833179285091e5201f5854
4,PROTECT YOUR INFORMATION AND YOUR COMPUTER,don <apf@wu-wien.ac.at>,b'ATTENTION: This is a MUST for ALL Computer U...,1,00373.ebe8670ac56b04125c25100a36ab0510
...,...,...,...,...,...
9348,RE: [Razor-users] What's wrong with the Razor ...,"""Rose, Bobby"" <brose@med.wayne.edu>","b""I'm one of the 30,000 but it's not working v...",0,00609.dd49926ce94a1ea328cce9b62825bc97
9349,Re: W3C approves HTML 4 'emotitags' [...],harley@argote.ch,"b""Damien Morton quoted:\n>W3C approves HTML 4 ...",0,00957.e0b56b117f3ec5f85e432a9d2a47801f
9350,Re: Ximian apt repos?,Mark Derricutt <mark@talios.com>,"b'On Mon, 2002-07-22 at 06:50, che wrote:\n\n>...",0,01127.841233b48eceb74a825417d8d918abf8
9351,Re: Installing RPM,Matthias Saou <matthias@egwn.net>,"b'Once upon a time, Manfred wrote :\n\n> I wou...",0,01178.5c977dff972cd6eef64d4173b90307f0


In [39]:
spam_df = df[df["isSpam"] == 1]

In [42]:
pd.set_option('display.max_colwidth', None)

IndexError: single positional indexer is out-of-bounds

In [32]:
spam_file = "/Users/satvikajmera/Documents/SMU MSDS Course Material/05-Summer 2022/DS 7333 - QtW/Case Studies - on file/Case Study 03/Satvik/SpamAssassinMessages/easy_ham_2/00747.352d424267d36975a7b40b85ffd0885e"

In [24]:
spam_filepath= "/Users/satvikajmera/Documents/SMU MSDS Course Material/05-Summer 2022/DS 7333 - QtW/Case Studies - on file/Case Study 03/Satvik/SpamAssassinMessages/spam/"

In [41]:
with open(spam_file) as f:
   soup = BeautifulSoup(f, "html.parser")

In [40]:
from bs4 import BeautifulSoup

with open(spam_file, 'r') as f:

    contents = f.read()

    soup = BeautifulSoup(contents, 'html')

    print(soup.h2)
    print(soup.head)
    print(soup.li)

None
None
None


In [29]:
with open(spam_file, 'rb') as fp:
    msg = BytesParser(policy=default).parse(fp)

# Now the header items can be accessed as a dictionary, and any non-ASCII will
# be converted to unicode:
print('To:', msg['to'])
print('From:', msg['from'])
print('Subject:', msg['subject'])
print('Body:', msg['body'])

To: social@linux.ie
From: Slim n Trim <yenene@mx2.1premio.com>
Subject: [ILUG-Social] re: Guaranteed to lose 10-12 lbs in 30 days 10.148
Body: None


In [28]:
msg.keys()

['Return-Path',
 'Delivered-To',
 'Received',
 'Received',
 'Received',
 'Received',
 'Received',
 'X-Authentication-Warning',
 'Received',
 'From',
 'To',
 'Date',
 'Message-Id',
 'MIME-Version',
 'Content-Type',
 'Content-Transfer-Encoding',
 'X-Mailer',
 'Thread-Index',
 'Content-Class',
 'X-Mimeole',
 'X-Originalarrivaltime',
 'Subject',
 'Sender',
 'Errors-To',
 'X-Mailman-Version',
 'Precedence',
 'List-Id',
 'X-Beenthere']

In [39]:
target = df['isSpam']
train = df[['Subject','From','Body']]
stop = stopwords.words('english')
df['Clean_Body'] = df['Body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df.head()


,Subject,From,Body,isSpam,Clean_Body
9348,Preferred Non-Smoker Rates for Smokers,IQ - TBA <tba@insiq.us>,Preferred Non-Smoker Just what the doc...,1,Preferred Non-Smoker Just doctor ordered! Case...
9349,"How to get 10,000 FREE hits per day to any web...",Mike <raye@yahoo.lv>,"Dear Subscriber, If I could show you a way t...",1,"Dear Subscriber, If I could show way get 10,00..."
9350,Cannabis Difference,"""Mr. Clean"" <cweqx@dialix.oz.au>",****Mid-Summer Customer Appreciation SALE!***...,1,****Mid-Summer Customer Appreciation SALE!****...
9351,[ILUG] WILSON KAMELA,"""wilsonkamela400@netscape.net"" <wilsonkamela50...",ATTN:SIR/MADAN S...,1,ATTN:SIR/MADAN STRICTLY CONFIDENTIAL. I please...
9352,NaN,NaN,mv 00001.317e78fa8ee2f54cd4890fdc09ba8176 000...,1,mv 00001.317e78fa8ee2f54cd4890fdc09ba8176 0000...


In [40]:
len(target) == len(train)

True

In [41]:
X_train, X_test, y_train, y_test = train_test_split(train, target,test_size=0.35, shuffle=True, random_state=42)

In [42]:
cv = CountVectorizer()
cv_data_train = cv.fit_transform(X_train)
cv_data_test = cv.transform(X_test)
model = MultinomialNB()
model.fit(cv_data_train, y_train)
cv_preds = model.predict(cv_data_test)